#ADNIMERGE Data Cleaning

* encode values: any string values need to be **hot encoded,** and numeric values need to be **normalized**


### Columns We're Choosing To ignore:
* colprot -- because it is the protocol under which the data is collected
* origprot -- original protocol the subject entered the study under
* ptid -- keeping to identify the patient as long as possible, then throwing out during actual training
* viscode -- the visit code
* SITE
* fieldstreng and FLDSTRENG_bl -- 1.5 or 3 Tesla MRI strength -- ignoring for now
* FSVERSION and FSVERSION_bl -- something about FreeSurver Version 4.3, 5.1, or 6.0 -- ignore
* update_stamp -- a datetime object

In [ ]:
# Imports
import numpy as np
import pandas as pd
from google.colab import drive
from sklearn.model_selection import train_test_split

In [ ]:
drive.mount('/content/drive')
full_df = pd.read_csv("/content/drive/MyDrive/Data_Science_Alzheimers_ADNI/ADNIMERGE_08Feb2024.csv")

# select the baseline entries for each patient (yields 2430 patients)
baseline_df = full_df[full_df['VISCODE'] == 'bl']

# make a copy of the baseline_df (to avoid View vs copy issues)
baseline_df_cleaned = baseline_df.copy()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-31-d6a9f224c94b>:2: DtypeWarning: Columns (19,20,21,50,51,104,105,106) have mixed types. Specify dtype option on import or set low_memory=False.
  full_df = pd.read_csv("/content/drive/MyDrive/Data_Science_Alzheimers_ADNI/ADNIMERGE_08Feb2024.csv")


In [ ]:
# print column labels nicely
i = 1
for col in sorted(full_df.columns):
  print(col, end="  ")
  if i % 7 == 0:
    print('')
  i = i+1

ABETA  ABETA_bl  ADAS11  ADAS11_bl  ADAS13  ADAS13_bl  ADASQ4  
ADASQ4_bl  AGE  APOE4  AV45  AV45_bl  CDRSB  CDRSB_bl  
COLPROT  DIGITSCOR  DIGITSCOR_bl  DX  DX_bl  EXAMDATE  EXAMDATE_bl  
EcogPtDivatt  EcogPtDivatt_bl  EcogPtLang  EcogPtLang_bl  EcogPtMem  EcogPtMem_bl  EcogPtOrgan  
EcogPtOrgan_bl  EcogPtPlan  EcogPtPlan_bl  EcogPtTotal  EcogPtTotal_bl  EcogPtVisspat  EcogPtVisspat_bl  
EcogSPDivatt  EcogSPDivatt_bl  EcogSPLang  EcogSPLang_bl  EcogSPMem  EcogSPMem_bl  EcogSPOrgan  
EcogSPOrgan_bl  EcogSPPlan  EcogSPPlan_bl  EcogSPTotal  EcogSPTotal_bl  EcogSPVisspat  EcogSPVisspat_bl  
Entorhinal  Entorhinal_bl  FAQ  FAQ_bl  FBB  FBB_bl  FDG  
FDG_bl  FLDSTRENG  FLDSTRENG_bl  FSVERSION  FSVERSION_bl  Fusiform  Fusiform_bl  
Hippocampus  Hippocampus_bl  ICV  ICV_bl  IMAGEUID  IMAGEUID_bl  LDELTOTAL  
LDELTOTAL_BL  M  MMSE  MMSE_bl  MOCA  MOCA_bl  MidTemp  
MidTemp_bl  Month  Month_bl  ORIGPROT  PIB  PIB_bl  PTAU  
PTAU_bl  PTEDUCAT  PTETHCAT  PTGENDER  PTID  PTMARRY  PTRACCAT  
RAVLT_

In [ ]:
# Remove Alligators Function
def remove_alligators(val):
  if(isinstance(val, float)):
    # no problem return
    return val

  # try to replace the alligators
  val = val.replace('>','')
  val = val.replace('<','')

  return val

# normalize Function
def basic_normalize(col_name):
  # get mean
  mean = baseline_df_cleaned[col_name].mean()

  # fill missing with mean
  baseline_df_cleaned[col_name].fillna(value=mean, inplace=True)

  # normalize
  abs_max = abs(baseline_df_cleaned[col_name].max())
  baseline_df_cleaned[col_name] = baseline_df_cleaned[col_name] / abs_max

# Hot Encode Function
def hot_encode(categories):
  my_dict = {}

  i=0
  for c in categories:
    my_dict[c] = i
    i = i+1

  return my_dict

In [ ]:
# Cleaning the 'ABETA' column which has numeric data with some problem values like >1700

baseline_df_cleaned['ABETA'] = baseline_df['ABETA'].apply(remove_alligators)

# try to cast the column to float
baseline_df_cleaned['ABETA'] = baseline_df_cleaned['ABETA'].astype(np.float64)

# calculate the mean
abeta_mean = baseline_df_cleaned['ABETA'].mean()

# replace nan with the mean
baseline_df_cleaned['ABETA'].fillna(value=abeta_mean, inplace=True)

# normalize
print("Before Normalization")
print(baseline_df_cleaned['ABETA'].describe())

abs_max = abs(baseline_df_cleaned['ABETA'].max())

baseline_df_cleaned['ABETA'] = baseline_df_cleaned.ABETA / abs_max

print("After Normalization")
print(baseline_df_cleaned['ABETA'].describe())

Before Normalization
count    2430.000000
mean      979.928642
std       323.299213
min       200.000000
25%       854.200000
50%       979.928642
75%       979.928642
max      1700.000000
Name: ABETA, dtype: float64
After Normalization
count    2430.000000
mean        0.576429
std         0.190176
min         0.117647
25%         0.502471
50%         0.576429
75%         0.576429
max         1.000000
Name: ABETA, dtype: float64


In [ ]:
# clean ABETA_bl column
baseline_df_cleaned['ABETA_bl'] = baseline_df['ABETA_bl'].apply(remove_alligators)

# try to cast the column to float
baseline_df_cleaned['ABETA_bl'] = baseline_df_cleaned['ABETA_bl'].astype(np.float64)

# calculate the mean
abeta_mean = baseline_df_cleaned['ABETA_bl'].mean()

# replace nan with the mean
baseline_df_cleaned['ABETA_bl'].fillna(value=abeta_mean, inplace=True)

# normalize
abs_max = abs(baseline_df_cleaned['ABETA_bl'].max())
baseline_df_cleaned['ABETA_bl'] = baseline_df_cleaned.ABETA_bl / abs_max


In [ ]:
# Clean 'TAU' column (problem like >1300)

baseline_df_cleaned['TAU'] = baseline_df['TAU'].apply(remove_alligators)

# try to cast the column to float
baseline_df_cleaned['TAU'] = baseline_df_cleaned['TAU'].astype(np.float64)

# get the mean
tau_mean = baseline_df_cleaned['TAU'].mean()

# replace nan with mean
baseline_df_cleaned['TAU'].fillna(value=tau_mean, inplace=True)

# normalize
abs_max = abs(baseline_df_cleaned['TAU'].max())
baseline_df_cleaned['TAU'] = baseline_df_cleaned.TAU / abs_max

In [ ]:
# clean TAU_bl column

baseline_df_cleaned['TAU_bl'] = baseline_df['TAU_bl'].apply(remove_alligators)

# try to cast the column to float
baseline_df_cleaned['TAU_bl'] = baseline_df_cleaned['TAU_bl'].astype(np.float64)

# get the mean
tau_mean = baseline_df_cleaned['TAU_bl'].mean()

# replace nan with mean
baseline_df_cleaned['TAU_bl'].fillna(value=tau_mean, inplace=True)

# normalize
abs_max = abs(baseline_df_cleaned['TAU_bl'].max())
baseline_df_cleaned['TAU_bl'] = baseline_df_cleaned.TAU_bl / abs_max

In [ ]:
# clean PTAU column
baseline_df_cleaned['PTAU'] = baseline_df['PTAU'].apply(remove_alligators)

# cast as float
baseline_df_cleaned['PTAU'] = baseline_df_cleaned['PTAU'].astype(np.float64)

#get mean
ptau_mean = baseline_df_cleaned['PTAU'].mean()

# replace nan with mean
baseline_df_cleaned['PTAU'].fillna(value=ptau_mean, inplace=True)

# normalize
abs_max = abs(baseline_df_cleaned['PTAU'].max())
baseline_df_cleaned['PTAU'] = baseline_df_cleaned.PTAU / abs_max

In [ ]:
# clean PTAU_bl column
baseline_df_cleaned['PTAU_bl'] = baseline_df['PTAU_bl'].apply(remove_alligators)

# cast as float
baseline_df_cleaned['PTAU_bl'] = baseline_df_cleaned['PTAU_bl'].astype(np.float64)

#get mean
ptau_mean = baseline_df_cleaned['PTAU_bl'].mean()

# replace nan with mean
baseline_df_cleaned['PTAU_bl'].fillna(value=ptau_mean, inplace=True)

# normalize
abs_max = abs(baseline_df_cleaned['PTAU_bl'].max())
baseline_df_cleaned['PTAU_bl'] = baseline_df_cleaned.PTAU_bl / abs_max

In [ ]:
# Hot Encode Patient Gender
categories = baseline_df_cleaned['PTGENDER'].unique()
my_dict = hot_encode(categories)

baseline_df_cleaned['PTGENDER'] = baseline_df_cleaned['PTGENDER'].replace(my_dict)

baseline_df_cleaned['PTGENDER'].unique()

array([0, 1])

In [ ]:
# normalize patient education
print("Before:", baseline_df_cleaned['PTEDUCAT'].unique())
basic_normalize('PTEDUCAT')
print("After:", baseline_df_cleaned['PTEDUCAT'].unique())

Before: [16 18 10 13 12  9 14 17 20 19 15  8  6  7 11  4]
After: [0.8  0.9  0.5  0.65 0.6  0.45 0.7  0.85 1.   0.95 0.75 0.4  0.3  0.35
 0.55 0.2 ]


In [ ]:
# hot encode ethnicity
categories = baseline_df_cleaned['PTETHCAT'].unique()
my_dict = hot_encode(categories)

baseline_df_cleaned['PTETHCAT'] = baseline_df_cleaned['PTETHCAT'].replace(my_dict)

baseline_df_cleaned['PTETHCAT'].unique()


array([0, 1, 2])

In [ ]:
# hot encode race
categories = baseline_df_cleaned['PTRACCAT'].unique()
my_dict = hot_encode(categories)

baseline_df_cleaned['PTRACCAT'] = baseline_df_cleaned['PTRACCAT'].replace(my_dict)

baseline_df_cleaned['PTRACCAT'].unique()

array([0, 1, 2, 3, 4, 5, 6])

In [ ]:
# hot encode marital status at baseline
categories = baseline_df_cleaned['PTMARRY'].unique()
my_dict = hot_encode(categories)

baseline_df_cleaned['PTMARRY'] = baseline_df_cleaned['PTMARRY'].replace(my_dict)

baseline_df_cleaned['PTMARRY'].unique()

array([0., 1., 2., 3., 4., 5.])

In [ ]:
# APOE4 (Apolipoprotein E4) Genotype has nan values.
# ASSUME: fill with zero
baseline_df_cleaned['APOE4'] = baseline_df_cleaned['APOE4'].replace(np.nan, 0)


In [ ]:
# Clean FDG (average FDG-PET of angular, temporal and posterior cingulate)
# ASSUME: fill with zero
baseline_df_cleaned['FDG'] = baseline_df_cleaned['FDG'].replace(np.nan, 0)

In [ ]:
# clean PIB (average PIB SUVR of frontal cortex, anterior cingulate, precuneus cortex, and parietal cortex)
# ASSUME: fill with zero
baseline_df_cleaned['PIB'].unique()
baseline_df_cleaned['PIB'] = baseline_df_cleaned['PIB'].replace(np.nan, 0)

In [ ]:
# clean AV45 (Average AV45 SUVR of frontal, anterior cingulate, precuneus, and parietal cortex relative to the cerebellum)
baseline_df_cleaned['AV45'].unique()
baseline_df_cleaned['AV45'] = baseline_df_cleaned['AV45'].replace(np.nan,0)

In [ ]:
# CDRSB (no cleaning needed)
baseline_df_cleaned['CDRSB'].unique()

array([ 0. ,  4.5,  1. ,  0.5,  6. ,  5. ,  4. ,  2.5,  1.5,  2. ,  3. ,
        8. ,  3.5,  7. ,  6.5,  5.5,  9. , 10. ])

In [ ]:
# ADAS11 Normalize
print("Before:", baseline_df_cleaned['ADAS11'].unique())
basic_normalize('ADAS11')
print("After:", baseline_df_cleaned['ADAS11'].unique())

Before: [10.67 22.   14.33  8.67 18.67 27.33 12.33  4.33  7.   10.33  6.67  4.
 10.   21.33  6.   11.33  8.33  3.   15.33  8.    9.67 18.33 20.33 15.67
  4.67 14.    5.67  2.    5.33 14.67 19.   18.    1.67 24.67  2.67 16.
 22.67 26.67 13.   11.67 12.67  9.33  5.   20.67  7.33 34.33 15.   12.
 16.33 13.67 23.   24.    6.33 13.33 28.    0.    9.    7.67  2.33 16.67
  3.67 19.67  1.33  3.33 17.   19.33 25.33 17.67 21.67 21.   23.33 11.
  1.   29.33 17.33 23.67 27.67 26.   22.33 20.   35.   25.67 25.   30.33
 37.   34.67 36.   35.33 33.   29.   27.   30.   40.   38.   31.     nan
 32.   32.33  0.67 42.67 28.33 24.33 26.33 33.33 30.67 28.67 36.33]
After: [0.25005859 0.51558472 0.33583314 0.20318725 0.43754394 0.64049684
 0.2889618  0.10147645 0.16404968 0.24209046 0.15631591 0.09374268
 0.23435669 0.49988282 0.14061401 0.26552613 0.19521912 0.07030701
 0.35926881 0.18748535 0.22662292 0.42957581 0.47644715 0.36723693
 0.10944457 0.32809937 0.13288024 0.04687134 0.12491212 0.34380127
 0.445

In [ ]:
# ADAS13 Normalize
print("Before:", baseline_df_cleaned['ADAS13'].unique())
basic_normalize('ADAS13')
print("After:", baseline_df_cleaned['ADAS13'].unique())

Before: [18.67 31.   21.33 14.67 25.67 40.33 24.33  8.33  9.   14.33  9.67  8.
 15.   34.33 16.33 12.33 12.    5.    7.   10.   23.33 11.   17.67 28.33
 32.33  3.   19.67 19.    4.   15.33  6.    9.33 24.67 27.   21.   28.
 23.    1.67 28.67 36.67  3.67 26.    5.33 20.33 38.67  2.67 26.67 16.
 21.67 13.33 16.67 31.67 22.33 17.33 49.33  8.67 24.   18.33  6.67 20.
 22.67 33.   15.67 10.67 19.33 35.   10.33 12.67 36.    1.   11.67 14.
  4.33 11.33 20.67  4.67 34.   29.67  3.33 22.   30.67 25.33 23.67 13.
  7.33  5.67 27.33 36.33 29.   18.   13.67 34.67   nan  7.67 30.   26.33
 25.   43.33 29.33 35.67 30.33 39.67 38.    2.   27.67 17.   31.33 33.67
 35.33 32.    6.33 47.   42.   37.67 40.   33.33 40.67 42.33 41.   49.
 47.67 50.   45.33 37.   46.   51.   39.   45.   48.   43.    0.   32.67
 50.33 52.    0.67 54.67  1.33  2.33 41.33 41.67 38.33 47.33 44.67 42.67
 48.33]
After: [0.34150357 0.5670386  0.39015914 0.2683373  0.46954454 0.73769892
 0.44503384 0.15236876 0.16462411 0.26211816 0.1

In [ ]:
# MMSE normalize
print("Before:", baseline_df_cleaned['MMSE'].unique())
basic_normalize('MMSE')
print("After:", baseline_df_cleaned['MMSE'].unique())

Before: [28. 20. 27. 29. 25. 24. 30. 26. 21. 23. 22. 19. 18. 16. 17. nan]
After: [0.93333333 0.66666667 0.9        0.96666667 0.83333333 0.8
 1.         0.86666667 0.7        0.76666667 0.73333333 0.63333333
 0.6        0.53333333 0.56666667 0.91269384]


In [ ]:
#RAVLT (forgetting) normalize
print("Before:", baseline_df_cleaned['RAVLT_forgetting'].unique())
basic_normalize('RAVLT_forgetting')
print("After:", baseline_df_cleaned['RAVLT_forgetting'].unique())

Before: [  6.   4.   5.   3.   1.   7.   2.   8.   0.  -1.  -5.  -2.   9.  12.
  10.  13.  nan  11.  15.  -3.  -4.  14. -19.]
After: [ 0.4         0.26666667  0.33333333  0.2         0.06666667  0.46666667
  0.13333333  0.53333333  0.         -0.06666667 -0.33333333 -0.13333333
  0.6         0.8         0.66666667  0.86666667  0.28637993  0.73333333
  1.         -0.2        -0.26666667  0.93333333 -1.26666667]


In [ ]:
# RAVLT Immediate normalize
print("Before:", baseline_df_cleaned['RAVLT_immediate'].unique())
basic_normalize('RAVLT_immediate')
print("After:", baseline_df_cleaned['RAVLT_immediate'])


Before: [44. 22. 37. 30. 17. 20. 45. 50. 40. 53. 42. 15. 27. 29. 61. 24. 41. 26.
 48. 47. 36. 32. 34. 49. 33. 18. 23. 57. 25. 54. 60. 28. 51. 14. 19. 21.
 35. 39. 11. 31. 38. 16. 56. 46. 67. 59. 52. 43. 13. 62. 55. 64. 12.  9.
 nan 10. 69. 66. 68.  7. 65. 58. 70. 63.  8. 71.  0.  3.  5.  1.]
After: 0        0.619718
1        0.309859
5        0.521127
10       0.521127
15       0.422535
           ...   
16237    0.309859
16257    0.436620
16326    0.633803
16338    0.676056
16408    0.516504
Name: RAVLT_immediate, Length: 2430, dtype: float64


In [ ]:
# encode the diagnosis as has alzheimers/MCI(value of 1) or not (value of 0)
dx_dict = {
    'CN':0, # normal (no AD)
    'Dementia':1, # assume yes AD
    'AD':1, # assume yes AD
    'MCI':1 # assume yes AD
}

baseline_df_cleaned['DX'] = baseline_df_cleaned['DX'].replace(dx_dict)

# drop nan rows where there isn't a diagnosis
baseline_df_cleaned = baseline_df_cleaned.dropna(subset=['DX'])

In [ ]:
# FAQ normalize
baseline_df_cleaned['FAQ'].unique()

array([ 0., 10., 17., 11.,  1.,  2., 12.,  7.,  3.,  9., 29., 20., 28.,
        4.,  5., 13.,  8., 22., 24.,  6., 14., 18., 15., 16., 21., 19.,
       25., 26., 23., 27., nan, 30.])

## Put it all together

In [ ]:
keep_cols = ['DX','ABETA', 'TAU', 'PTAU', 'ABETA_bl', 'TAU_bl', 'PTAU_bl',
             'PTGENDER', 'PTEDUCAT', 'PTETHCAT','PTRACCAT', 'PTMARRY', 'APOE4',
             'FDG', 'PIB', 'AV45','CDRSB', 'AV45', 'CDRSB', 'ADAS11', 'ADAS13',
             'MMSE', 'RAVLT_forgetting', 'RAVLT_immediate']
baseline_df_all_cleaned = baseline_df_cleaned[keep_cols]

# split data into training and test
train, test = train_test_split(baseline_df_all_cleaned, random_state=104, test_size=0.30, shuffle=True)

# save train and test to csv
train.to_csv(r'/content/drive/MyDrive/adnimerge_08Feb2024_cleaned_train_2.csv')
test.to_csv(r'/content/drive/MyDrive/adnimerge_08Feb2024_cleaned_test_2.csv')
# !mv /content/drive/MyDrive/adnimerge_08Feb2024_cleaned_train_2.csv /content/drive/MyDrive/Data_Science_Alzheimers_ADNI
# !mv /content/drive/MyDrive/adnimerge_08Feb2024_cleaned_test_2.csv /content/drive/MyDrive/Data_Science_Alzheimers_ADNI

mv: cannot stat 'adnimerge_08Feb2024_cleaned_train_2.csv': No such file or directory
mv: cannot stat 'adnimerge_08Feb2024_cleaned_test_2.csv': No such file or directory
